<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import numpy as np


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping


In [4]:
url = "https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv"

df = pd.read_csv(url)

In [5]:
df.isnull().any()

customerID          False
gender              False
SeniorCitizen       False
Partner             False
Dependents          False
tenure              False
PhoneService        False
MultipleLines       False
InternetService     False
OnlineSecurity      False
OnlineBackup        False
DeviceProtection    False
TechSupport         False
StreamingTV         False
StreamingMovies     False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges        False
Churn               False
dtype: bool

In [6]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,6172-FECYY,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [7]:
df['MultipleLines'].unique()

array(['No phone service', 'No', 'Yes'], dtype=object)

In [8]:
df['OnlineSecurity'].unique()

array(['No', 'Yes', 'No internet service'], dtype=object)

In [9]:
df['TotalCharges'].value_counts()

           11
20.2       11
19.75       9
19.9        8
19.65       8
           ..
55.45       1
3769.7      1
4816.7      1
2043.45     1
418.4       1
Name: TotalCharges, Length: 6531, dtype: int64

In [10]:
df['TotalCharges'] = df['TotalCharges'].replace(' ',0)

In [11]:
df['TotalCharges'] = df['TotalCharges'].astype('float')

In [12]:
df['MonthlyCharges'] = df['MonthlyCharges'].astype('float')

In [13]:
df = df.drop('customerID', axis=1)

In [14]:
df['Churn'] = df['Churn'].replace(["Yes", "No"], [1,0])

In [15]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [16]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df)

In [17]:
from category_encoders import OneHotEncoder

In [18]:
X_train = train.drop('Churn', axis=1)
y_train = train['Churn']
X_test = test.drop('Churn', axis=1)
y_test = test['Churn']

In [19]:
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [20]:
X_train_encoded = np.asarray(X_train_encoded)

In [21]:
X_train_encoded.shape

(5282, 45)

In [22]:
X_test_encoded = np.asarray(X_test_encoded)

In [23]:
X_test_encoded.shape

(1761, 45)

In [24]:
y_train.shape

(5282,)

In [25]:
model = Sequential()
model.add(Dense(1280, input_dim=X_train_encoded.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
stop = EarlyStopping(min_delta=0.01, patience=5)

In [30]:
model.fit(X_train_encoded, y_train, epochs=100, validation_data=(X_test_encoded, y_test), callbacks=[stop])

Train on 5282 samples, validate on 1761 samples
Epoch 1/100
5282/5282 [==============================] - 3s 654us/sample - loss: 11.7126 - accuracy: 0.6925 - val_loss: 4.1034 - val_accuracy: 0.7814
Epoch 2/100
5282/5282 [==============================] - 1s 98us/sample - loss: 4.9496 - accuracy: 0.7236 - val_loss: 0.8308 - val_accuracy: 0.7127
Epoch 3/100
5282/5282 [==============================] - 1s 106us/sample - loss: 2.5546 - accuracy: 0.7283 - val_loss: 2.6528 - val_accuracy: 0.7768
Epoch 4/100
5282/5282 [==============================] - 1s 98us/sample - loss: 2.0921 - accuracy: 0.7298 - val_loss: 0.9768 - val_accuracy: 0.7888
Epoch 5/100
5282/5282 [==============================] - 1s 106us/sample - loss: 1.6424 - accuracy: 0.7401 - val_loss: 1.9275 - val_accuracy: 0.7859
Epoch 6/100
5282/5282 [==============================] - 1s 101us/sample - loss: 1.0205 - accuracy: 0.7501 - val_loss: 0.5615 - val_accuracy: 0.7944
Epoch 7/100
5282/5282 [==============================] - 1s

In [37]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, Nadam

In [32]:
def create_model():
    model = Sequential()
    model.add(Dense(1280, input_dim=X_train_encoded.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [33]:
model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test))

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7955317735671997 using {'batch_size': 40, 'epochs': 20}
Means: 0.7873908877372742, Stdev: 0.003875575790969396 with: {'batch_size': 10, 'epochs': 20}
Means: 0.791367256641388, Stdev: 0.013306001450683759 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7955317735671997, Stdev: 0.009333955366213956 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7870205044746399, Stdev: 0.02046840076884167 with: {'batch_size': 60, 'epochs': 20}
Means: 0.793827223777771, Stdev: 0.005432767424473814 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7754628181457519, Stdev: 0.020939970274908393 with: {'batch_size': 100, 'epochs': 20}


In [36]:
def create_model(learn_rate=0.01):
    model = Sequential()
    model.add(Dense(1280, input_dim=X_train_encoded.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr=learn_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=20)
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test))

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7962897062301636 using {'learn_rate': 0.001}
Means: 0.7962897062301636, Stdev: 0.013116698147933595 with: {'learn_rate': 0.001}
Means: 0.7887184262275696, Stdev: 0.007758082742536382 with: {'learn_rate': 0.01}
Means: 0.7349504351615905, Stdev: 0.016510696290566153 with: {'learn_rate': 0.1}
Means: 0.7375994443893432, Stdev: 0.0192777945310452 with: {'learn_rate': 0.2}
Means: 0.7410110354423523, Stdev: 0.020452163944339914 with: {'learn_rate': 0.3}


In [30]:
def create_model(dropout=0.2):
    model = Sequential()
    model.add(Dense(1280, input_dim=X_train_encoded.shape[1], activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer= Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=20)
dropout = [0.001, 0.01, 0.05, 0.1, 0.2, 0.3]
param_grid = dict(dropout=dropout)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test))

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7909913301467896 using {'dropout': 0.01}
Means: 0.7498939156532287, Stdev: 0.07343254571325372 with: {'dropout': 0.001}
Means: 0.7909913301467896, Stdev: 0.017980516205921088 with: {'dropout': 0.01}
Means: 0.7902292847633362, Stdev: 0.019640841780465452 with: {'dropout': 0.05}
Means: 0.7902328491210937, Stdev: 0.017843529406174938 with: {'dropout': 0.1}
Means: 0.7883448362350464, Stdev: 0.019289308568417405 with: {'dropout': 0.2}
Means: 0.7760297536849976, Stdev: 0.0174534716276342 with: {'dropout': 0.3}


In [31]:
def create_model(first_neuron=1280, second_neuron=128):
    model = Sequential()
    model.add(Dense(first_neuron, input_dim=X_train_encoded.shape[1], activation='relu'))
    model.add(Dropout(0.01))
    model.add(Dense(second_neuron, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer= Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=20)

param_grid = {'first_neuron': [1280, 128, 64], 'second_neuron': [1280, 128, 64, 32]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test))

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7962906002998352 using {'first_neuron': 128, 'second_neuron': 1280}
Means: 0.7442129969596862, Stdev: 0.06411225013021271 with: {'first_neuron': 1280, 'second_neuron': 1280}
Means: 0.784935736656189, Stdev: 0.028263784444725017 with: {'first_neuron': 1280, 'second_neuron': 128}
Means: 0.7926948070526123, Stdev: 0.012416622366303717 with: {'first_neuron': 1280, 'second_neuron': 64}
Means: 0.7675075531005859, Stdev: 0.030026671823730408 with: {'first_neuron': 1280, 'second_neuron': 32}
Means: 0.7962906002998352, Stdev: 0.010809697170568873 with: {'first_neuron': 128, 'second_neuron': 1280}
Means: 0.7726213693618774, Stdev: 0.028334709106895558 with: {'first_neuron': 128, 'second_neuron': 128}
Means: 0.7253089070320129, Stdev: 0.07286045676232301 with: {'first_neuron': 128, 'second_neuron': 64}
Means: 0.7788724422454834, Stdev: 0.010932264294302751 with: {'first_neuron': 128, 'second_neuron': 32}
Means: 0.7894763588905335, Stdev: 0.016271260998943184 with: {'first_neuron': 64, 'se

In [32]:
def create_model(first_activation='relu', second_activation='relu', output_activation='sigmoid'):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train_encoded.shape[1], activation=first_activation))
    model.add(Dropout(0.01))
    model.add(Dense(1280, activation=second_activation))
    model.add(Dense(1, activation=output_activation))
    optimizer= Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=20)

param_grid = {'first_activation': ['relu', 'selu', 'sigmoid', 'softmax'], 'second_activation': ['relu', 'selu', 'sigmoid', 'softmax'],
             'output_activation': ['sigmoid', 'softmax']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test))

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7932642340660095 using {'first_activation': 'relu', 'output_activation': 'sigmoid', 'second_activation': 'relu'}
Means: 0.7932642340660095, Stdev: 0.017174931428814384 with: {'first_activation': 'relu', 'output_activation': 'sigmoid', 'second_activation': 'relu'}
Means: 0.7841724276542663, Stdev: 0.017478329475160696 with: {'first_activation': 'relu', 'output_activation': 'sigmoid', 'second_activation': 'selu'}
Means: 0.7712979435920715, Stdev: 0.007611246939509721 with: {'first_activation': 'relu', 'output_activation': 'sigmoid', 'second_activation': 'sigmoid'}
Means: 0.7510471343994141, Stdev: 0.023260056596903625 with: {'first_activation': 'relu', 'output_activation': 'sigmoid', 'second_activation': 'softmax'}
Means: 0.26467256546020507, Stdev: 0.007847475302483675 with: {'first_activation': 'relu', 'output_activation': 'softmax', 'second_activation': 'relu'}
Means: 0.26467256546020507, Stdev: 0.007847475302483675 with: {'first_activation': 'relu', 'output_activation': 'soft

In [35]:
def create_model(optimizer=Adam):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train_encoded.shape[1], activation='relu'))
    model.add(Dropout(0.01))
    model.add(Dense(1280, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=20)

optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test))

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7951552987098693 using {'optimizer': 'Nadam'}
Means: 0.735327422618866, Stdev: 0.007847466487768823 with: {'optimizer': 'SGD'}
Means: 0.7419852614402771, Stdev: 0.10555805411319939 with: {'optimizer': 'RMSprop'}
Means: 0.6870762407779694, Stdev: 0.12150998907767549 with: {'optimizer': 'Adagrad'}
Means: 0.7198154091835022, Stdev: 0.04452029959153313 with: {'optimizer': 'Adadelta'}
Means: 0.7824650168418884, Stdev: 0.027311715354244603 with: {'optimizer': 'Adam'}
Means: 0.7184749603271484, Stdev: 0.04665520606889455 with: {'optimizer': 'Adamax'}
Means: 0.7951552987098693, Stdev: 0.012791488040536597 with: {'optimizer': 'Nadam'}


In [38]:
def create_model():
    model = Sequential()
    model.add(Dense(128, input_dim=X_train_encoded.shape[1], activation='relu'))
    model.add(Dropout(0.01))
    model.add(Dense(1280, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer= Nadam(lr=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

stop = EarlyStopping(min_delta=0.01, patience=5)

model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=40, epochs=20)


param_grid = {'epochs': [20, 40, 80, 100, 400, 1000]}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test), callbacks=[stop])

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7968589663505554 using {'epochs': 100}
Means: 0.7883362293243408, Stdev: 0.01619376162337619 with: {'epochs': 20}
Means: 0.7187041282653809, Stdev: 0.12319231362364273 with: {'epochs': 40}
Means: 0.7516318082809448, Stdev: 0.08542275738049718 with: {'epochs': 80}
Means: 0.7968589663505554, Stdev: 0.012163361496566704 with: {'epochs': 100}
Means: 0.7116377711296081, Stdev: 0.1051862895482901 with: {'epochs': 400}
Means: 0.7565202116966248, Stdev: 0.060921503115243476 with: {'epochs': 1000}


In [ ]:
#Best model according to grid searches
model = Sequential()
model.add(Dense(128, input_dim=X_train_encoded.shape[1], activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(1280, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
optimizer= Nadam(lr=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
stop = EarlyStopping(min_delta=0.01, patience=5)
model.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test), callbacks=[stop])

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?